In [1]:
# Data processing
import pandas as pd
import numpy as np
import functools

# Set pandas to show all columns in the dataframes
pd.set_option("display.max_columns", None)

# Global Constants
STRUCT_PROGRESS_SIZE = 504
STRUCT_PROGRESS_ADD = 0

In [2]:
data = pd.read_csv("outputs/ProgressMap.csv", sep=';', index_col="Index")
data

,Bit Index,Description,Category,BitStart,Address,Stride,Row,Column,Byte Offset,Code Name
Index,,,,,,,,,,
1,113,"""Completed: Setting Up Camp: Ancient Forest I""",Delivery,1,+18*0+E,24,0,14,14,delivery_completed_setting_up_camp_ancient_for...
2,115,"""Completed: Setting Up Camp: Ancient Forest II""",Delivery,3,+18*0+E,24,0,14,14,delivery_completed_setting_up_camp_ancient_for...
3,121,"""Completed: Setting Up Camp: Coral Highlands""",Delivery,1,+18*0+F,24,0,15,15,delivery_completed_setting_up_camp_coral_highl...
4,351,"""Obtained: A Great Help""",Delivery,7,+18*1+13,24,1,19,43,delivery_obtained_a_great_help
5,352,"""Completed: A Great Help""",Delivery,0,+18*1+14,24,1,20,44,delivery_completed_a_great_help
...,...,...,...,...,...,...,...,...,...,...
296,3928,"""Fatalis ? +""",Layered,0,+18*14+B,24,20,11,491,layered_fatalis_alpha_plus
297,3929,"""Fatalis ? +""",Layered,1,+18*14+B,24,20,11,491,layered_fatalis_beta_plus
298,3930,"""Kulve Taroth ? +""",Layered,2,+18*14+B,24,20,11,491,layered_kulve_taroth_alpha_plus


In [3]:
f = open('outputs/progress.bt', 'w')
unknown_index = 0
unknown_consecutive = 0

for y in range(STRUCT_PROGRESS_SIZE):
    in_byte = data[data["Byte Offset"] == y]
    if in_byte.shape[0] > 0:
        if unknown_consecutive == 1:
            print(f"u8 unknown{unknown_index};", file=f)
        elif unknown_consecutive > 1:
            print(f"u8 unknown{unknown_index}[{unknown_consecutive}];", file=f)
        unknown_index += int(unknown_consecutive > 0)
        unknown_consecutive = 0
        
        bit_unknown_index = 0
        bit_unknown_consecutive = 0
        bit_unknown_found = False
        bits = []
        
        for x in range(8):
            bit = in_byte[in_byte["BitStart"] == x]
            bit_unknown_index_chr = chr(ord('a') + bit_unknown_index)
                
            if (bit.shape[0] == 1):
                bit = bit.iloc[0]
                if bit_unknown_consecutive > 0:
                    print(f"u8 unknown{unknown_index}{bit_unknown_index_chr} : {bit_unknown_consecutive};", file=f)
                    bit_unknown_index += 1
                    bit_unknown_found = True
                code_name = bit.at["Code Name"]
                bit_length = 1
                print(f"u8 {code_name} : {bit_length};", file=f)
                bit_unknown_consecutive = 0
            else:
                bit_unknown_consecutive += 1
            
            if x == 7 and bit_unknown_consecutive > 0:
                print(f"u8 unknown{unknown_index}{bit_unknown_index_chr} : {bit_unknown_consecutive};", file=f)
                bit_unknown_found = True
                bit_unknown_consecutive = 0
        if bit_unknown_found:
            unknown_index += 1
    else:
        unknown_consecutive += 1
    if y == STRUCT_PROGRESS_SIZE - 1 and unknown_consecutive > 0:
        print(f"u8 unknown{unknown_index}[{unknown_consecutive}];", file=f)
        

f.close()